<a href="https://colab.research.google.com/github/cedro3/yolact_edge/blob/master/YolactEdge_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# Githubからコードをコピー、重みをダウンロード
!git clone https://github.com/cedro3/yolact_edge.git
!git clone https://github.com/chentinghao/download_google_drive.git

%cd yolact_edge
!mkdir -p weights

!python ../download_google_drive/download_gdrive.py 1EAzO-vRDZ2hupUJ4JFSUi40lAZ5Jo-Bp ./weights/yolact_edge_54_800000.pth
!python ../download_google_drive/download_gdrive.py 15jyd5CRJxNiA41UMjGbaSnmaytfeILfI ./calib_images_coco.zip
!unzip -q calib_images_coco.zip

In [ ]:
# ライブラリー・インポート
from eval import *
from utils.logging_helper import setup_logger
from google.colab.patches import cv2_imshow
from video_2_images import video_2_images
parse_args(["--config=yolact_edge_config", "--calib_images=../calib_images"])
from eval import args

setup_logger(logging_level=logging.INFO)
logger = logging.getLogger("yolact.eval")

In [ ]:
# 学習済みモデル構築
args.trained_model = "./weights/yolact_edge_54_800000.pth"
args.yolact_transfer = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')
logger.info('Loading model...')
net = Yolact(training=False)
net.load_weights(args.trained_model, args=args)
net.eval()
logger.info('Model loaded.')

net.detect.use_fast_nms = args.fast_nms
cfg.mask_proto_debug = args.mask_proto_debug

args.score_threshold = 0.15 
args.top_k = 15

# 静止画から物体検出する

In [ ]:
# サンプル画像を確認する
im = cv2.imread("./pic/sample.jpg")
cv2_imshow(im)

In [ ]:
# 静止画から物体検出
frame = torch.from_numpy(im).cuda().float()
batch = FastBaseTransform()(frame.unsqueeze(0))

extras = {"backbone": "full", "interrupt": False, "keep_statistics": False,
          "moving_statistics": None}

with torch.no_grad():
    preds = net(batch, extras=extras)["pred_outs"]

In [ ]:
# 予測結果を見る
dets = preds[0]
print('Scores:', dets['score'][:args.top_k])
print('Classes:', dets['class'][:args.top_k])
print('Boxes:', dets['box'][:args.top_k])

In [ ]:
# 物体検出した画像を表示
img_numpy = prep_display(preds, frame, None, None, undo_transform=False)
cv2_imshow(img_numpy)

# ビデオから物体検出する

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode
 
mp4 = open('./video/sample.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
# 指定したビデオを静止画にバラして images フォルダーに保存する
video_2_images('./video/sample.mp4')

In [ ]:
# 連続して静止画から物体検出
import glob
from tqdm import tqdm

files=[]
for name in sorted(glob.glob('./images/*.jpg')):
     files.append(name)

for file in tqdm(files):
     img = cv2.imread(file)
     frame = torch.from_numpy(img).cuda().float()
     batch = FastBaseTransform()(frame.unsqueeze(0))

     extras = {"backbone": "full", "interrupt": False, "keep_statistics": False,
               "moving_statistics": None}

     with torch.no_grad():
          preds = net(batch, extras=extras)["pred_outs"]

     img_numpy = prep_display(preds, frame, None, None, undo_transform=False)
     cv2.imwrite(file, img_numpy)

In [ ]:
# 連続した静止画からmp4動画を作成する（既に output.mp4 があれば削除する）
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

!ffmpeg -r 30 -i images/%06d.jpg -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode
 
mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")